## Testando as informações do CSV movies

In [11]:
import pandas as pd
from datetime import datetime

movies_imdb = pd.read_csv("movies.csv", sep='|')

movies_imdb = movies_imdb.drop_duplicates(subset=['id'])

movies_imdb['genero'] = movies_imdb['genero'].replace('\\N', '')

movies_imdb['anoLancamento'] = movies_imdb['anoLancamento'].replace('\\N', pd.NA)

total_sem_genero = movies_imdb['genero'].eq('').sum()

total_sem_ano_lancamento = movies_imdb['anoLancamento'].isnull().sum()

total_filmes = len(movies_imdb)

movies_imdb['anoLancamento'] = pd.to_datetime(movies_imdb['anoLancamento'], errors='coerce')

ano_atual = datetime.now().year
filmes_filtrados = movies_imdb[
    (movies_imdb['genero'].str.contains('Drama', regex=True, na=False)) &
    (~movies_imdb['anoLancamento'].isna()) & 
    (movies_imdb['anoLancamento'].dt.year >= ano_atual - 2)
]

total_drama_romance_ultimos_10_anos = len(filmes_filtrados)

print("Total de filmes sem gênero:", total_sem_genero)
print("Total de filmes sem ano de lançamento:", total_sem_ano_lancamento)
print("Total de filmes no conjunto de dados:", total_filmes)
print("Total de filmes do gênero Drama dos últimos 2 anos:", total_drama_romance_ultimos_10_anos)


/var/folders/5q/4krf7jmj6fgfll_b9kgmff9r0000gn/T/ipykernel_7391/2941513771.py:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_imdb = pd.read_csv("movies.csv", sep='|')


Total de filmes sem gênero: 8511
Total de filmes sem ano de lançamento: 40
Total de filmes no conjunto de dados: 244543
Total de filmes do gênero Drama dos últimos 2 anos: 2737


## Testando a API do TMDB

In [12]:
import requests
import apikey
import json

API_KEY = apikey.api_key

resultados = []

for index, row in filmes_filtrados.iterrows():
    movie_id = row['id']
    
    url_external_ids = f"https://api.themoviedb.org/3/movie/{movie_id}/external_ids?api_key={API_KEY}&language=pt-BR"

    response_external_ids = requests.get(url_external_ids)
    data_external_ids = response_external_ids.json()

    if response_external_ids.status_code == 200:
        if 'imdb_id' in data_external_ids:
            imdb_id = data_external_ids['imdb_id']

            url_movie_details = f"https://api.themoviedb.org/3/movie/{imdb_id}?api_key={API_KEY}&language=pt-BR&external_source=imdb_id"

            response_movie_details = requests.get(url_movie_details)
            data_movie_details = response_movie_details.json()

            if response_movie_details.status_code == 200:
                resultados.append(data_movie_details)
            else:
                print(f"Falha na solicitação para obter detalhes do filme {movie_id}.")
        else:
            print(f"ID IMDb não encontrado nos IDs externos do filme {movie_id}.")
    else:
        print(f"Falha na solicitação para obter IDs externos do filme {movie_id}. Código de status:", response_external_ids.status_code)

with open('resultados_filmes.json', 'w') as file:
    json.dump(resultados, file, indent=4)

print("Resultados salvos com sucesso no arquivo 'resultados_filmes.json'.")


Falha na solicitação para obter IDs externos do filme tt10054946. Código de status: 404
Falha na solicitação para obter IDs externos do filme tt10188844. Código de status: 404
Falha na solicitação para obter IDs externos do filme tt10231812. Código de status: 404
Falha na solicitação para obter IDs externos do filme tt10474784. Código de status: 404
Falha na solicitação para obter IDs externos do filme tt10481912. Código de status: 404
Falha na solicitação para obter IDs externos do filme tt10545214. Código de status: 404
Falha na solicitação para obter IDs externos do filme tt10651304. Código de status: 404
Falha na solicitação para obter IDs externos do filme tt10829654. Código de status: 404


KeyboardInterrupt: 

## Testando arquivo json

In [12]:
import json

with open('filmes.json', 'r') as file:
    data = json.load(file)

quantidade_ids = len(data)

print(f"A quantidade de IDs no JSON é: {quantidade_ids}")


A quantidade de IDs no JSON é: 2410


In [14]:

dados_filmes = []

for filme in data:
    dados_filme = {}
    
    if 'title' in filme:
        dados_filme['nome'] = filme['title']
    if 'genres' in filme and len(filme['genres']) > 0:
        dados_filme['genero'] = ', '.join([genero['name'] for genero in filme['genres']])
    if 'vote_average' in filme:
        dados_filme['avaliacao'] = filme['vote_average']
    if 'budget' in filme:
        dados_filme['orcamento'] = filme['budget']
    
    dados_filmes.append(dados_filme)

for filme in dados_filmes:
    print(filme)


{'nome': 'Predestinado: Arigó e o Espírito do Dr. Fritz', 'genero': 'Drama', 'avaliacao': 7.267, 'orcamento': 0}
{'nome': 'Bilanggo Sa Dilim', 'genero': 'Thriller, Drama, Crime', 'avaliacao': 0.0, 'orcamento': 0}
{'nome': 'Wielka droga', 'genero': 'Drama, Guerra', 'avaliacao': 5.0, 'orcamento': 0}
{'nome': '5-25-77', 'genero': 'Drama, Comédia', 'avaliacao': 6.65, 'orcamento': 0}
{'nome': 'Hamara Khandaan', 'genero': 'Drama, Romance', 'avaliacao': 10.0, 'orcamento': 0}
{'nome': 'Ten Tricks', 'genero': 'Drama, Comédia', 'avaliacao': 4.333, 'orcamento': 0}
{'nome': 'Invasion of the Not Quite Dead', 'avaliacao': 10.0, 'orcamento': 0}
{'nome': 'Transe', 'genero': 'Drama', 'avaliacao': 1.0, 'orcamento': 0}
{'nome': 'लाल सिंह चड्ढा', 'genero': 'Drama, Romance', 'avaliacao': 6.483, 'orcamento': 23000000}
{'nome': 'Escuridão', 'genero': 'Drama, Guerra, Terror', 'avaliacao': 6.643, 'orcamento': 0}
{'nome': 'La casa entre los cactus', 'genero': 'Drama', 'avaliacao': 6.533, 'orcamento': 0}
{'nome'